# Install requirements

In [2]:
%pip install tensorflow_addons numpy pandas tensorflow sklearn nltk spacy textblob gensim scipy seaborn matplotlib minio mlflow wordcloud boto3

     |████████████████████████████████| 1.1 MB 21.2 MB/s            
     |████████████████████████████████| 497.6 MB 83.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.5 MB 77.2 MB/s            
     |████████████████████████████████| 6.2 MB 79.6 MB/s            
     |████████████████████████████████| 636 kB 76.7 MB/s            
     |████████████████████████████████| 24.1 MB 46.9 MB/s            
     |████████████████████████████████| 292 kB 79.7 MB/s            
     |████████████████████████████████| 371 kB 65.4 MB/s            
     |████████████████████████████████| 14.5 MB 82.6 MB/s            
     |████████████████████████████████| 65 kB 64.6 MB/s            
     |████████████████████████████████| 1.4 MB 75.2 MB/s            
     |████████████████████████████████| 5.8 MB 79.3 MB/s            
     |████████████████████████████████| 126 kB 83.3 MB/s            
     |████████████████████████████████| 462 kB 5.3 MB/s    

In [3]:
import os
cwd = os.getcwd()
import sys
# sys.path is a list of absolute path strings
sys.path.append('/opt/app-root/src/anz_ml_project/')
from src.loadingdata.read_dataset import ReadData
from src.features.build_features import BuildFeatures
from src.modules.build_model import BuildModel
from src.modules.train_model import MLflow, TrainModel
from src.modules.predict_model import Predictor,Transformer,DownloadArtifact


import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from textblob import TextBlob
from nltk.stem import PorterStemmer,SnowballStemmer
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
# from wordcloud import WordCloudfrom wordcloud import WordCloud
from io import StringIO
import string
import gensim
from gensim.models import Word2Vec
import itertools
import scipy
from scipy import spatial
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import joblib

import mlflow
import warnings

from minio import Minio
import subprocess
import ipynbname
warnings.filterwarnings("ignore")
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)


tokenizer = ToktokTokenizer()

2022-04-22 06:15:56.924620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 06:15:56.924697: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Initialize the config file for mlflow and Minio

In [4]:

HOST = "http://mlflow:5500"

PROJECT_NAME = "NlpTc"
EXPERIMENT_NAME = "NlpLstm"

os.environ['MLFLOW_S3_ENDPOINT_URL']='http://minio-ml-workshop:9000'
os.environ['AWS_ACCESS_KEY_ID']='minio'
os.environ['AWS_SECRET_ACCESS_KEY']='minio123'
os.environ['AWS_REGION']='us-east-1'
os.environ['AWS_BUCKET_NAME']='raw-data-saeed'



## Define a Function to read from Minio S3 Bucket

In [6]:
def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient
client = get_s3_server()

## SetUp MLFlow to track the model

In [7]:
mlflow = MLflow(mlflow, HOST,EXPERIMENT_NAME).SetUp_Mlflow()

2022/04/22 06:16:54 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


# Readinng the data


In [9]:
train_data, test_data, train_labels, test_labels,enc = ReadData(S3BucketName = "raw-data-saeed",FILE_NAME="data.csv").ReadDataFrameData()

['Bank account or service' 'Consumer Loan' 'Credit card'
 'Credit reporting' 'Debt collection' 'Money transfers' 'Mortgage'
 'Other financial service' 'Payday loan' 'Prepaid card' 'Student loan']
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([119,  90, 184, 242, 436,  22, 400,   2,  26,  10,  69]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 30,  22,  46,  61, 109,   5, 100,   1,   7,   2,  17]))


In [10]:
joblib.dump(enc, 'labelencoder.pkl')  

['labelencoder.pkl']

In [11]:
# from src.features.build_features import BuildFeatures
train_data.shape

test_labels.shape


(400,)

# Prepare data for modeling

In [12]:
BFCLASS = BuildFeatures(TRAIN_DATA=train_data,TEST_DATA=test_data,TRAIN_LABELS=train_labels,TEST_LABELS=test_labels, GloveData="glove.6B.50d.txt",EMBEDDING_DIM=50, WEIGHT_FLAG = False,MLFLOW_S3_ENDPOINT_URL = "minio-ml-workshop:9000",AWS_ACCESS_KEY_ID='minio',AWS_SECRET_ACCESS_KEY = 'minio123',SECURE = False)

In [13]:
train_data,test_data,train_labels , test_labels,word_index,tokenizer,MAX_SEQUENCE_LENGTH = BFCLASS.PreProcessingTextData()

Maximum Sequence length is 348 .
Found 7366 unique tokens.
Shape of train data tensor: (1600, 348)
Shape of train label tensor: (1600, 11)
Shape of test label tensor: (400, 11)


In [14]:
joblib.dump(tokenizer, 'tokenizer.pkl')  

['tokenizer.pkl']

# Deep Learning define, train and test model


## Define the Model

In [15]:
model = BuildModel(WORD_INDEX=word_index, MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH, EMWEIGHTS=[]).SetupModel()

2022-04-22 06:18:04.129378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-22 06:18:04.129425: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-22 06:18:04.129467: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterhub-nb-opentlc-2dmgr): /proc/driver/nvidia/version does not exist
2022-04-22 06:18:04.129751: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 348, 50)           368350    
                                                                 
 bidirectional (Bidirectiona  (None, 348, 200)         120800    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              935936    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 11)                5643      
                                                                 
Total params: 1,430,729
Trainable params: 1,430,729
Non-trainable params: 0
_________________________________________________________________


## Train the Model

In [16]:
model, history= TrainModel(model, tokenizer, enc,train_data, train_labels,test_data, test_labels,HOST, EXPERIMENT_NAME, BATCH_SIZE=64,EPOCHS=2).ModelTraining()

2022/04/22 06:20:08 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/5
25/25 [==============================] - 166s 6s/step - loss: 1.9714 - acc: 0.3250 - val_loss: 1.6113 - val_acc: 0.4800
Epoch 2/5
25/25 [==============================] - 162s 6s/step - loss: 1.4740 - acc: 0.4925 - val_loss: 1.3340 - val_acc: 0.5375
Epoch 3/5
25/25 [==============================] - 160s 6s/step - loss: 1.2385 - acc: 0.5638 - val_loss: 1.3274 - val_acc: 0.5425
Epoch 4/5
25/25 [==============================] - 159s 6s/step - loss: 1.0642 - acc: 0.6363 - val_loss: 1.2053 - val_acc: 0.5725
Epoch 5/5
25/25 [==============================] - 156s 6s/step - loss: 0.8877 - acc: 0.7000 - val_loss: 1.1786 - val_acc: 0.6100


2022-04-22 06:33:41.457681: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp9265dz5e/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp9265dz5e/model/data/model/assets


### Plot the training and testing Loss

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :RNN - LSTM',fontsize=16)
plt.show()

#### Plot the training and testing Accuracy

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training acc', 'Validation acc'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves :RNN - LSTM',fontsize=16)
plt.show()

## Test the model

In [ ]:
#predictions on test data
predicted=model.predict(test_data)
predicted

In [ ]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels, predicted.round())
print('precision: \n{}'.format(precision))
print('recall: \n{}'.format(recall))
print('fscore: \n{}'.format(fscore))
print('support: \n{}'.format(support))
print("############################")

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(test_labels.argmax(axis=1), predicted.argmax(axis=1))
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="BuPu",xticklabels=enc.classes_,yticklabels=enc.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()